# **역대박스오피스 순위 TOP 100 크롤링 (네이버)**

## *1. 데이터 수집을 위한 라이브러리 설치, 임포트 및 설정*

### 데이터 수집을 위한 라이브러리 설치

In [ ]:
!pip install requests
!pip install openpyxl
!pip install pyperclip
!pip install selenium
!pip install pandas
!apt install chromium-chromedriver    # 크롬드라이버 설치

  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=b660ee1cd04ce86d12a2d04c50f6d82779909cde41dde428b8157480ecf729b5
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip
     |████████████████████████████████| 958 kB 11.0 MB/s 
     |████████████████████████████████| 356 kB 63.2 MB/s 
     |████████████████████████████████| 138 kB 66.4 MB/s 
     |████████████████████████████████| 55 kB 2.5 MB/s 
     |████████████████████████████████| 3.6 MB 63.7 MB/s 
     |████████████████████████████████| 54 kB 3.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urlli

### 데이터 수집을 위한 라이브러리 임포트

In [ ]:
import requests
from openpyxl.workbook import Workbook
import time # time.sleep에 사용할 함수
import pyperclip
import re
import selenium
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


### 크롬 드라이버 설정

In [ ]:
options = webdriver.ChromeOptions() # ChromeOptions()를 만들어 add_argument를 통해 Headless모드, 크롬 창의 크기, gpu(그래픽카드 가속)를 사용 여부 등의 옵션을 설정.
options.add_argument('--headless') # 크롬이 Headless모드로 동작하도록 만들어주는 키워드
options.add_argument("--start-fullscreen") # 브라우저가 풀스크린 모드(F11)로 실행
options.add_argument("window-size=1920x1080") # 크롬 창의 크기를 직접 지정해 준 이유는? 일반적으로 노트북이나 데스크탑에서 사용하는 모니터의 해상도가 1920x1080이기 때문이다.
options.add_argument("disable-gpu") # gpu를 사용하지 않도록 지정, 코드를 실행했을 때 GPU 에러가 난다면 --disable-gpu 를 사용

# GUI 제공이 불가능한 상황에서 headless와 함께 사용
options.add_argument("no-sandbox") 
options.add_argument("--disable-dev-shm-usage")

## *2. 영화 정보를 담기위한 변수 생성*

### 모든 영화 정보를 담을 딕셔너리 변수 생성

In [ ]:
# 아래의 변수에 담긴 데이터들을 컬럼별로 한번에 묶어주기 위한 변수
movie_collect = {}

### 각 영화의 정보를 담을 리스트 변수들 생성

In [ ]:
title_infos = [] # 제목
content_infos = [] # 줄거리
genre_infos = [] # 장르
score_infos = [] # 평점
date_infos = [] # 개봉일

## *3. 영화 정보 긁어오기*

### 변수 초기화

In [ ]:
# 변수에 값이 담겨있을 수 있으므로 변수를 초기화 후 크롤링을 진행한다
title_infos.clear()
content_infos.clear()
genre_infos.clear()
score_infos.clear()
date_infos.clear()

### 가상의 드라이버 생성

In [ ]:
# driver 변수를 만들 때 단순하게 chromedriver의 위치만 적어주는 것이 아니라 chrome_options라는 이름의 인자를 함께 넘겨줘야한다.
# 크롬 드라이버 설정에서 지정해준 옵션들을 가상의 driver를 생성할 때 전달
driver = webdriver.Chrome("chromedriver", options=options)

# 불러올 페이지 주소를 url 변수에 담는다
url = "https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&qvt=0&query=%EC%97%AD%EB%8C%80%EC%98%81%ED%99%94%EC%88%9C%EC%9C%84"

# 크롬 드라이를 사용하여 url 변수에 담긴 주소를 불러온다
driver.get(url) 

# 가상의 크롬 브라우저가 실행되었는지 스크린샷한 이미지 파일을 저장
driver.get_screenshot_as_file('create_headless_driver.png')

# 2초의 시간 간격을 가짐
time.sleep(2)

### 영화 상세보기 페이지가 맞는지 확인하는 변수 생성 <br> (평점 정보를 가져올 때 한번 사용)


In [ ]:
def check_exists_by_xpath(ch_driver, xpath):
    try:
        ch_driver.find_element_by_xpath(xpath) # html의 xpath 를 이용해 원하는 값을 긁어온다. 뒤에 .text를 입력하면 그중 텍스트만 읽어올 수 있다
    except NoSuchElementException: # 지정한 요소가 없을 때, 즉 아직 웹페이지가 준비가 지정되어 있지 않은 상황에서 크롤링을 진행할 경우 발생 
        return False
    return True

### 영화 정보를 긁어오는 반복문 생성

In [ ]:
#첫번째 반복문 시작-----------------------------------------------------------------------------------------------------------------
# 영화 상세보기 페이지 이동
for i in range(1, 14): # 총 13페이지를 반복하기위한 for문, 총 페이지수 변경 시 14 대신 마지막 페이지 수 입력
  page_cnt = str(i) # 순차적으로 가져오는 1~13 중 하나의 값을 page_cnt 변수에 문자형으로 담는다

  # 8개의 영화를 가져오기 위한 반복문 생성을 위해 two_range에 9를 담는다
  two_range = 9 # 8개의 영화를 긁어오기 위한 값을 변수에 담는다

#두번째 반복문 시작-----------------------------------------------------------------------------------------------------------------
  # 페이지를 반복문을 사용하여 넘기며 데이터를 긁어오기 위한 for문 생성           
  for j in range(1, two_range): 
    # 위에 if문 제거 후 try문 추가
    try:
        movie_cnt = str(j) # 순차적으로 가져오는 1~8(또는 1~4) 중 하나의 값을 movie_cnt 변수에 문자형으로 담는다

        # xpath를 지정해서 가상의 크롬드라이버가 경로를 찾을 수 있게 함
        # 영화의 제목이 담겨있는 경로를 지정하여 temp_title 변수에 담아준다, 알라딘의 상세 페이지 구조가 다르기 때문에 알라딘을 구분하기 위해서 가져온다
        temp_title = driver.find_element_by_xpath('//*[@id="main_pack"]/div[2]/div/div[2]/div/div[1]/div[2]/div[' + page_cnt + ']/ul/li[' + movie_cnt + ']/div[2]/a/div/strong').text
        print(temp_title)

        # 영화의 이미지를 클릭하여 상세페이지로 이동
        movie = driver.find_element_by_xpath('//*[@id="main_pack"]/div[2]/div/div[2]/div/div[1]/div[2]/div[' + page_cnt + ']/ul/li[' + movie_cnt + ']/div[1]/a/span[1]').click() # 클릭을 실행하는 함수
        time.sleep(4) # 4초의 시간 간격을 가짐

          
        # //*[@id="main_pack"]/div[3]/div[1]/div[1]/h2/a/strong <- 알라딘 div[3]
        # //*[@id="main_pack"]/div[2]/div[1]/div[1]/h2/a/strong <- 다른 영화 div[2]
        basic_num = '2'
        # 알라딘과 다른 영화의 xpath 값이 다르므로 if 문으로 알라딘의 영화 상세보기를 눌렀을 때 위의 xpath 경로를 지정해주기 위해  basic_num에 3을 담아준다
        if temp_title == '알라딘':
            basic_num = '3'

        # 영화 제목 수집
        try: # 영화 상세보기 페이지에 영화제목 위치에 영화제목이 있을 경우 text를 추출하여 movie_title 변수에 담는다. <-- 알라딘을 제외한 영화들
            movie_title = driver.find_element_by_xpath('//*[@id="main_pack"]/div[' + basic_num + ']/div[1]/div[1]/h2/a/strong').text
        except: # 없을 경우 temp_title에 담긴 값을 movie_title에 담는다. <- 알라딘
            movie_title = temp_title

        # 평점
        if check_exists_by_xpath(driver, '//*[@id="main_pack"]/div[' + basic_num + ']/div[2]/div[1]/div[2]/div[2]/dl/div[3]/dd',): # 영화 상세보기 페이지에 평점 위치가 존재할 경우 위에서 정의한 함수에서 True 값을 받아 if문 수행
            movie_score = driver.find_element_by_xpath('//*[@id="main_pack"]/div[' + basic_num + ']/div[2]/div[1]/div[2]/div[2]/dl/div[3]/dd').text # 영화 상세보기 페이지에 평점 위치에 평점이 있을 경우 text를 추출하여 movie_score 변수에 담는다
        else: # 누락값 NaN은 '데이터 자체'가 없음을 의미한다. 그렇기에 '같다'의 의미 자체가 성립되지 않는 존재이다. 이러한 정의가 가능하기에 누락값은 자기 자신과 비교해도 Boolean Type의 값은 False로 출력된다.
            # 없을 경우
            movie_score = np.nan # numpy 라이브러리를 사용하여 movie_score에 nan값을 담아준다.

        # 장르 수집
        # 장르는 필요없는 값을 제거하는 '정규식' 과정을 진행함  > 장르의 값만 가져올 수 있게 함
        # < 액션한국118분 >의 값에서  => < 액션 > 이라는 장르만 추출 (나머지 값들은 제거)
        try: # 영화 상세보기 페이지에 장르 위치에 장르가 있을 경우 text를 추출하여 movie_genre에 변수에 담는다
            movie_genre = driver.find_element_by_xpath('//*[@id="main_pack"]/div[' + basic_num + ']/div[2]/div[1]/div[2]/div[2]/dl/div[1]/dd').text

            # 1) 기본적인 전처리
            movie_genre = movie_genre.replace(',' , '').replace(' ','') #  공백 및 (,) 콤마 제거

            # 2) 정규식을 사용한 전처리 
            movie_genre2 = re.sub(r'[0-9]+분', '', movie_genre).strip() # movie_genre 의 상영시간과 상영시간 문자열에서 양쪽 끝에 있는 공백과 \n 기호를 삭제시켜 준 후 movie_genre2에 값을 담아준다.
            movie_genre3 = re.sub(r'(한국)|(미국)|(일본)|(캐나다)|(영국)|(스페인)|(중국)', '', movie_genre2) # movie_genre2 의 국가와 국가 문자열에서 양쪽 끝에 있는 공백과 \n 기호를 삭제시켜 준 후 movie_genre3에 값을 담아준다.
            movie_genre3 = movie_genre3.strip() 

        except: # 없을 경우
            movie_genre = np.nan # numpy 라이브러리를 사용하여 movie_genre에 nan값을 담아준다.

        # 줄거리 더보기 누르기
        try: # 영화 상세보기 페이지에 줄거리 위치의 마지막에에 더보기 링크가 있을 경우 클릭하여 줄거리 더보기 페이지로 이동한다
            more_content = driver.find_element_by_xpath('//*[@id="main_pack"]/div[' + basic_num + ']/div[2]/div[1]/div[2]/div[2]/div/a').click()
        except: # 없을 경우
            more_content = np.nan # numpy 라이브러리를 사용하여 more_content에 nan값을 담아준다.

        # 줄거리 수집
        try: # 줄거리 더보기 페이지에 줄거리 위치에 줄거리가 있을 경우 text를 추출하여 movie_content에 변수에 담는다
            movie_content = driver.find_element_by_xpath('//*[@id="main_pack"]/div[2]/div[2]/div[2]/div/div[2]/p').text
            time.sleep(3) # 3초의 시간 간격을 가짐
            driver.back() # 뒤로가기 버튼을 클릭하여 영화상세보기페이지로 이동
            time.sleep(2.5) # 2.5초의 시간 간격을 가짐

        except: # 없을 경우
            movie_content = np.nan # numpy 라이브러리를 사용하여 movie_content에 nan값을 담아준다.
        
            
        # 데이터 수집 잘 되고 있는지 확인하기 위해 print() 사용하여 출력
        print("=" *50 + "영화 데이터 수집 중입니다." + "="*50)
        print("수집한 제목: ", movie_title)
        print("수집한 장르: ", movie_genre3)
        print("수집한 평점: ", movie_score)
        print("수집한 줄거리: ", movie_content)
        print("="*120)

        # 긁어온 영화의 정보가 담긴 변수를 미리 생성해놓은 리스트형의 전역변수에 append()를 사용하여 요소로 추가시킨다
        title_infos.append(movie_title)
        genre_infos.append(movie_genre3)
        score_infos.append(movie_score)
        content_infos.append(movie_content)
        driver.back() # 뒤로가기 버튼을 클릭하여 영화목록페이지로 이동한다
        time.sleep(2) # 2초의 시간 간격을 가짐

        # 페이지 적힌 숫자의 -1번 만큼 페이지 이동하기 ex) 4페이지로 이동하려면 '>' 버튼 3번 클릭 필요    
        if i > 1: # 현재 반복문이 돌고있는 페이지(i)가 1보다 클 경우 if문 수행
            for click_one in range(1, i): # 1부터 i-1 까지의 범위를 반복
                print("----------------- 다음 페이지 클릭 >>>>> -----------------")
                next_page = driver.find_element_by_xpath('//*[@id="main_pack"]/div[2]/div/div[2]/div/div[1]/div[3]/div/a[2]').click() # 다음페이지로 넘어가는 버튼을 클릭
                time.sleep(0.7) # 0.7초의 시간 간격을 가짐
    # except문 추가
    except:
      print("<<<<<<<<<<<<<< 마지막 영화입니다 >>>>>>>>>>>>>>>") # 마지막 페이지에서 출력못한 영화 8-n개 만큼 출력
#두번째 반복문 끝-----------------------------------------------------------------------------------------------------------------
  print("******* 이 페이지는 수집 끝! 다음 페이지로! >> *******")
  next_page = driver.find_element_by_xpath('//*[@id="main_pack"]/div[2]/div/div[2]/div/div[1]/div[3]/div/a[2]').click() # 다음페이지로 넘어가는 버튼을 클릭
#첫번째 반복문 끝-----------------------------------------------------------------------------------------------------------------
driver.close()
print("======================== 역대흥행순위 TOP 100 영화 크롤링을 마쳤습니다.=========================================")


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


명량


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:66: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:72: DeprecationWarning: find_element_by_* comman

==================================================영화 데이터 수집 중입니다.==================================================
수집한 제목:  명량
수집한 장르:  액션
수집한 평점:  8.88
수집한 줄거리:  1597년 임진왜란 6년, 오랜 전쟁으로 인해 혼란이 극에 달한 조선. 무서운 속도로 한양으로 북상하는 왜군에 의해 국가존망의 위기에 처하자 누명을 쓰고 파면 당했던 이순신 장군(최민식)이 삼도수군통제사로 재임명된다. 하지만 그에게 남은 건 전의를 상실한 병사와 두려움에 가득 찬 백성, 그리고 12척의 배 뿐. 마지막 희망이었던 거북선마저 불타고 잔혹한 성격과 뛰어난 지략을 지닌 용병 구루지마(류승룡)가 왜군 수장으로 나서자 조선은 더욱 술렁인다. 330척에 달하는 왜군의 배가 속속 집결하고 압도적인 수의 열세에 모두가 패배를 직감하는 순간, 이순신 장군은 단 12척의 배를 이끌고 명량 바다를 향해 나서는데…! 12척의 조선 vs 330척의 왜군 역사를 바꾼 위대한 전쟁이 시작된다!
극한직업
==================================================영화 데이터 수집 중입니다.==================================================
수집한 제목:  극한직업
수집한 장르:  코미디
수집한 평점:  9.20
수집한 줄거리:  불철주야 달리고 구르지만 실적은 바닥, 급기야 해체 위기를 맞는 마약반! 더 이상 물러설 곳이 없는 팀의 맏형 고반장은 국제 범죄조직의 국내 마약 밀반입 정황을 포착하고 장형사, 마형사, 영호, 재훈까지 4명의 팀원들과 함께 잠복 수사에 나선다. 마약반은 24시간 감시를 위해 범죄조직의 아지트 앞 치킨집을 인수해 위장 창업을 하게 되고, 뜻밖의 절대미각을 지닌 마형사의 숨은 재능으로 치킨집은 일약 맛집으로 입소문이 나기 시작한다. 수사는 뒷전, 치킨장사로 눈코 뜰 새 없이 바빠진 마약반에게 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:107: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


도둑들
==================================================영화 데이터 수집 중입니다.==================================================
수집한 제목:  도둑들
수집한 장르:  범죄
수집한 평점:  7.64
수집한 줄거리:  한 팀으로 활동 중인 한국의 도둑 뽀빠이와 예니콜, 씹던껌, 잠파노. 미술관을 터는데 멋지게 성공한 이들은 뽀빠이의 과거 파트너였던 마카오박이 제안한 홍콩에서의 새로운 계획을 듣게 된다. 여기에 마카오박이 초대하지 않은 손님, 감옥에서 막 출소한 금고털이 팹시가 합류하고 5명은 각자 인생 최고의 반전을 꿈꾸며 홍콩으로 향한다. 홍콩에서 한국 도둑들을 기다리고 있는 4인조 중국도둑 첸, 앤드류, 쥴리, 조니. 최고의 전문가들이 세팅된 가운데 서로에 대한 경계를 늦추지 않는 한국과 중국의 도둑들. 팽팽히 흐르는 긴장감 속에 나타난 마카오박은 자신이 계획한 목표물을 밝힌다. 그것은 마카오 카지노에 숨겨진 희대의 다이아몬드 <태양의 눈물>. 성공을 장담할 수 없는 위험천만한 계획이지만 2천만 달러의 달콤한 제안을 거부할 수 없는 이들은 태양의 눈물을 훔치기 위한 작업에 착수한다. 그러나 진짜 의도를 알 수 없는 비밀스런 마카오박과 그런 마카오박의 뒤통수를 노리는 뽀빠이, 마카오박에게 배신당한 과거의 기억을 잊지 못하는 팹시와 팀보다 눈 앞의 현찰을 먼저 챙기는 예니콜, 그리고 한국 도둑들을 믿지 않는 첸과 중국 도둑들까지. 훔치기 위해 모였지만 목적은 서로 다른 10인의 도둑들은 서서히 자신만의 플랜을 세우기 시작하는데…
----------------- 다음 페이지 클릭 >>>>> -----------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:101: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


7번방의 선물
==================================================영화 데이터 수집 중입니다.==================================================
수집한 제목:  7번방의 선물
수집한 장르:  코미디
수집한 평점:  8.83
수집한 줄거리:  최악의 흉악범들이 모인 교도소 7번방에 이상한 놈이 들어왔다! 그는 바로 6살 지능의 딸바보 '용구'! 평생 죄만 짓고 살아온 7번방 패밀리들에게 떨어진 미션은 바로 '용구' 딸 '예승'이를 외부인 절대 출입금지인 교도소에 반.입.하.는.것! 2013년 새해, 웃음과 감동 가득한 사상초유의 합동작전이 시작된다!
----------------- 다음 페이지 클릭 >>>>> -----------------
알라딘
==================================================영화 데이터 수집 중입니다.==================================================
수집한 제목:  알라딘
수집한 장르:  모험
수집한 평점:  9.42
수집한 줄거리:  머나먼 사막 속 신비의 아그라바 왕국의 시대. 좀도둑 ‘알라딘’은 마법사 ‘자파’의 의뢰로 마법 램프를 찾아 나섰다가 주인에게 세 가지 소원을 들어주는 지니를 만나게 되고, 자스민 공주의 마음을 얻으려다 생각도 못했던 모험에 휘말리게 되는데…
----------------- 다음 페이지 클릭 >>>>> -----------------
암살
==================================================영화 데이터 수집 중입니다.==================================================
수집한 제목:  암살
수집한 장르:  액션
수집한 평점:  9.10
수집한 줄거리:  1933년 조국이 사라진 시대 대한민국 임시정부는 일본 측에 노출되지 않은 세 명을 암살작전에 지목한다. 한국 독립군 저격수 안

### 영화의 정보들을 담은 리스트 데이터들을 딕셔너리 변수의 해당 컬럼에 담기

In [ ]:
# movie_collect 딕셔너리 변수의 지정 key의 값에(지정 key 값이 없을 경우 생성) 영화정보가 담긴 리스트 변수의 데이터 담기    
movie_collect["제목"] = title_infos
movie_collect["장르"] = genre_infos
movie_collect["평점"] = score_infos
movie_collect["줄거리"] = content_infos

## *4. 크롤링한 영화 정보 csv 파일로 저장하기*

### 영화들의 정보가 담긴 딕셔너리 변수 DataFrame으로 변환

In [ ]:
# movie_collect 변수를 pandas의 DataFrame 형태로 변환하여 df 변수에 담아준다
df = pd.DataFrame({k: np.nan if not v else v for k, v in movie_collect.items()})

### 변수를 csv 파일로 저장

In [ ]:
# csv 파일로 저장하기
df.to_csv("영화_TOP100_final.csv", index=False, encoding='utf-8-sig')